In [29]:
import pandas as pd
from modules import lyric_scraper as ls
from tqdm import tqdm
import multiprocessing as mp
import numpy as np
import os

# enable tqdm for pandas
tqdm.pandas()

In [30]:
metadata = pd.read_csv('data/dataset_500.csv')
metadata

,Index,Artist,Title,Mood
0,ML728,Sam Cooke,Trouble Blues,angry
1,ML319,Ferry Corsten,Fire,angry
2,ML1971,Phyllis Hyman,"Loving You, Losing You",angry
3,ML388,Marilyn Manson,The Fight Song,angry
4,ML1908,Lindsey Buckingham,Go Insane,angry
...,...,...,...,...
495,ML1840,Slayer,Necrophobic,sad
496,ML603,Monchy & Alexandra,Dos Locos,sad
497,ML1939,Ruben Blades,El Padre Antonio Y El Monaguillo Andres,sad
498,ML57,Comeback Kid,Do Yourself A Favor,sad


In [32]:
def scrape_lyrics(row):
    search_name = row.Artist + " - " + row.Title
    Index = row.Index

    lyrics = ls.get_lyrics(search_name)

    # if lyrics are found, save them to a file
    if lyrics:
        with open(f'data/lyrics/{Index}.txt', 'w') as f:
            f.write(lyrics)
            return True
    else:
        return False

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = mp.Pool(n_cores)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

def apply_scrape_lyrics(df):
    return df.progress_apply(scrape_lyrics, axis=1)

# Assuming metadata is your DataFrame
# logs = parallelize_dataframe(metadata, apply_scrape_lyrics)

In [28]:
# read all lyrics
lyrics = []
path = "data/lyrics/"

names = [name.replace(".txt", "") for name in (os.listdir(path))]
names.sort()

# All indices in metadata that are not in names
missing_indices = metadata[~metadata.Index.isin(names)]
missing_indices

,Index,Artist,Title,Mood
380,ML1915,Underworld,Hamburg Hotel,sad


In [37]:
ls.get_lyrics("Underworld - Hamburg Hotel")

''